In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [3]:
try:
    df = pd.read_csv("flight_data_2024.csv")
    print("Arquivo carregado com sucesso!")

except FileNotFoundError:
    print("ERRO: Arquivo não encontrado")

print(f"Dataset carregado com {df.shape[0]} linhas e {df.shape[1]} colunas.")

Arquivo carregado com sucesso!
Dataset carregado com 1048575 linhas e 18 colunas.


### 2 - INSPEÇÃO E QUALIDADE DOS DADOS ###

In [ ]:
print("\n --- 2.1. Primeiras linhas ---")
display(df.head())

In [ ]:
try:
    df['flight_id'] = np.arange(df.shape[0]) + 1
    colunas = ['flight_id'] + [col for col in df.columns if col != 'flight_id']
    df = df[colunas]

    print(f"\n Coluna 'flight_id' criada com sucesso!")
    print(f"Total de {df.shape[0]:,} voos.")

    print("\n ---- 5 primeiras linhas do df com o ID novo ----")
    print(df[['flight_id', 'fl_date', 'origin', 'cancelled']].head(5))

except Exception as e:
    print(f"ocorreu um erro inesperado: {e}")

In [7]:
print(f"--- Descrição dos dados ---")
display(df.describe().T)

--- Descrição dos dados ---


,count,mean,std,min,25%,50%,75%,max
year,1048575.0,2024.000000,0.000000,2024.0,2024.0,2024.0,2024.0,2024.0
month,1048575.0,1.478081,0.499520,1.0,1.0,1.0,2.0,2.0
day_of_month,1048575.0,15.305120,8.585503,1.0,8.0,15.0,23.0,31.0
day_of_week,1048575.0,3.893483,2.010038,1.0,2.0,4.0,6.0,7.0
dep_time,1026022.0,1325.073671,497.299032,1.0,911.0,1323.0,1736.0,2400.0
taxi_out,1025450.0,18.250121,10.440251,1.0,12.0,15.0,21.0,213.0
wheels_off,1025450.0,1349.995608,498.042601,1.0,929.0,1337.0,1750.0,2400.0
wheels_on,1024898.0,1476.155761,519.868238,1.0,1058.0,1510.0,1914.0,2400.0
taxi_in,1024898.0,8.082517,6.512591,1.0,4.0,6.0,9.0,444.0
cancelled,1048575.0,0.022226,0.147419,0.0,0.0,0.0,0.0,1.0


In [6]:
print("\n --- 2.3. Contagem de valores nulos ---")
nulos = df.isnull().sum().sort_values(ascending=False)
nulos_percentual = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
df_nulos = pd.DataFrame({'Nulos': nulos, 'Percentual': nulos_percentual})
print(df_nulos[df_nulos['Nulos'] > 0]) # Exibe apenas colunas com nulos


 --- 2.3. Contagem de valores nulos ---
            Nulos  Percentual
air_time    25751    2.455809
wheels_on   23677    2.258017
taxi_in     23677    2.258017
taxi_out    23125    2.205374
wheels_off  23125    2.205374
dep_time    22553    2.150824


In [5]:
voos_cancelados = df[df['cancelled'] == 1].copy()
cidades_distintas = voos_cancelados['origin_city_name'].dropna().unique().tolist()
total_cidades = len(cidades_distintas)
total_cancelamentos = voos_cancelados['cancelled'].sum()

print(f'O total de cidades de origem distinctas: {total_cidades}')
print(f'O total de Voos cancelados é: {total_cancelamentos}')


O total de cidades de origem distinctas: 301
O total de Voos cancelados é: 23306


In [4]:
df['cancelled'].value_counts(normalize=True).to_frame('proporcao')

,proporcao
cancelled,
0,0.977774
1,0.022226


In [ ]:
# 2. Contagem das Observações por Mês
contagem_mensal = df['month'].value_counts().sort_index()

# 3. Criação do Gráfico de Barras
ax = sns.barplot(x=contagem_mensal.index, y=contagem_mensal.values, palette="viridis")


In [ ]:
cancelamentos_por_cidade = (
    df.groupby('origin_city_name')['cancelled'].sum().sort_values(ascending=False)
)

cidades_com_mais_cancelamentos = cancelamentos_por_cidade.head(15)
print(f"\n --- Gerando Gráfico: TOP 15 cidades por número de cancelamentos ---")


plt.figure(figsize=(14,7))
grafico_voos_cancelados = sns.barplot(
    x=cidades_com_mais_cancelamentos.index,
    y=cidades_com_mais_cancelamentos.values,
    palette="Reds_d"
)

# Adicionando Títulos e Rótulos
plt.title('Top 15 Cidades de Origem com Maior Número de Voos Cancelados', fontsize=16, fontweight='bold')
plt.xlabel('Cidade de Origem', fontsize=12)
plt.ylabel('Total de Voos Cancelados', fontsize=12)
plt.xticks(rotation=45, ha='right')


    # 1. Adicionar os valores exatos em cima de cada barra (como fizemos antes)
for p in grafico_voos_cancelados.patches:
        grafico_voos_cancelados.annotate(f'{int(p.get_height()):,}',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha = 'center', va = 'center',
                    xytext = (0, 9),
                    textcoords = 'offset points')

    # 2. Ajustar o layout
plt.tight_layout()

    # 3. Exibir o gráfico!
plt.show()

In [ ]:
df['total_minutos_atraso'] = df['weather_delay'] + df['late_aircraft_delay']
df['flag_atrasado'] = (df['total_minutos_atraso'] > 15).astype(int)
total_voos_atrasados = df['flag_atrasado'].sum()
print(f"O total de Voos atrasados é: {total_voos_atrasados}")

In [ ]:
atrasos_por_cidade = (
    df.groupby('origin_city_name')['flag_atrasado'].sum().sort_values(ascending=False)
    )

cidades_com_mais_atrasos = atrasos_por_cidade.head(15)
print(f"\n Gerando gráfico das TOP 15 cidades com mais atrasos")

plt.figure(figsize=(14, 7))
grafico_voos_atrasados = sns.barplot(
    x=cidades_com_mais_atrasos.index,
    y=cidades_com_mais_atrasos.values,
    palette="Reds_d"
)

# Adicionando Títulos e Rótulos
plt.title('Top 15 Cidades de Origem com Maior Número de Voos Atrasados', fontsize=16, fontweight='bold')
plt.xlabel('Cidade de Origem', fontsize=12)
plt.ylabel('Total de Voos Atrasados', fontsize=12)
plt.xticks(rotation=45, ha='right')

    # Linhas que faltam para finalizar: Adicionar anotações e exibir

    # 1. Adicionar os valores exatos em cima de cada barra (como fizemos antes)
for p in grafico_voos_atrasados.patches:
        grafico_voos_atrasados.annotate(f'{int(p.get_height()):,}',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha = 'center', va = 'center',
                    xytext = (0, 9),
                    textcoords = 'offset points')

    # 2. Ajustar o layout
plt.tight_layout()

    # 3. Exibir o gráfico!
plt.show()

In [ ]:
# Agrupa pela cidade e aplica as funções de agregação necessárias
df_resumo_cidade = df.groupby('origin_city_name').agg(
        # 1. Quantidade de Voos Agendados (simplesmente a contagem de observações)
        quantidade_voos_agendados=('year', 'count'),

        # 2. Quantidade de Voos Atrasados (soma do nosso indicador binário)
        quantidade_voos_atrasados=('flag_atrasado', 'sum'),

        # 4. Quantidade de Voos Cancelados (soma da coluna 'cancelled')
        quantidade_voos_cancelados=('cancelled', 'sum')

    ).reset_index() # Transforma o índice 'origin_city_name' em uma coluna

df_resumo_cidade['prop_atrasados'] = (
    df_resumo_cidade['quantidade_voos_atrasados'] / df_resumo_cidade['quantidade_voos_agendados']
)

df_resumo_cidade['prop_cancelados'] = (
    df_resumo_cidade['quantidade_voos_cancelados'] / df_resumo_cidade['quantidade_voos_agendados']
)

df_resumo_cidade = df_resumo_cidade.sort_values(
    by='quantidade_voos_agendados', ascending=False
)

df_resumo_cidade['prop_atrasados'] = df_resumo_cidade['prop_atrasados'].map('{:.2%}'.format)
df_resumo_cidade['prop_cancelados'] = df_resumo_cidade['prop_cancelados'].map('{:.2%}'.format)

print(f"Novo Dataframe criado com sucesso! Abaixo as top 15 cidades por volume de voos agendados")
display(df_resumo_cidade.head(15))

### Inferencias iniciais ###

- O Dataset refere-se ao histórico de voos de 2024, porém só possui informações de vôos nos meses de Janeiro e Fevereiro (1 e 2);
- As informaçoes nulas do dataset se concentram nas variáveis associadas a tempo.
-